In [ ]:
!git clone https://github.com/LinyingLyu/ChronoGPT.git
%cd ChronoGPT

# Load ChronoGPT-instruct

In [ ]:
import torch
import tiktoken
from huggingface_hub import hf_hub_download
from ChronoGPT_instruct import *
import warnings
warnings.filterwarnings("ignore")

from huggingface_hub import HfApi, login
login(token='Your token here')
api = HfApi()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")

# Load model
repo_id = "manelalab/chrono-gpt-instruct-v1-20241231"
config_path = hf_hub_download(repo_id=repo_id, filename="config.pt")
bin_path = hf_hub_download(repo_id=repo_id, filename="pytorch_model.bin")

config = torch.load(config_path, map_location='cpu')
model = ChronoGPT(**config).to(device).half()
model.load_state_dict(torch.load(bin_path, map_location=device))
model.eval()

print("ChronoGPT-instruct successfully loaded!")

# Instruction Following

In [ ]:
user_input = "Who is the first man landing on the moon?"
input_text = f"\n\n### Instruction:\n{user_input}\n\n### Response:\n"

token_ids = generate(
    model=model,
    idx=text_to_token_ids(input_text, tokenizer).to(device),
    max_new_tokens=128,
    context_size=1024,
    eos_id=50256
)

text = token_ids_to_text(token_ids, tokenizer)
response = extract_response(text, input_text)
print(f"You: {user_input}")
print(f"\nModel: {response}\n")